# 🎛️ Post-Mix Processing Pipeline (Ultra Clean)

**Dual-mode audio processing system:**
- 📄 **Single-file mode**: Traditional stereo mix processing
- 🎚️ **Stem mode**: 4-stem intelligent processing (drums, bass, vocals, music)

**Features:**
- 17 BIG variant profiles for different styles
- 4 mastering styles (neutral, warm, bright, loud)
- LUFS normalization for streaming
- Comprehensive analysis and reporting

## ⚙️ User Configuration
**Edit these settings, then run all cells**

In [1]:
# ============================================
# 🎛️ USER CONFIGURATION
# ============================================

# Processing Mode Selection
RUN_SINGLE_FILE = True      # Traditional single-file processing
RUN_STEM_MASTERING = True    # 4-stem intelligent processing (RECOMMENDED)

# Input Paths
BASE_INPUT_PATH = "/Users/itay/Documents/post_mix_data/professional_mixes/10_configurations_20250901_120434/01_Standard_Mix/"

# Single file path (if RUN_SINGLE_FILE = True)
MIX_SOURCE_PATH = BASE_INPUT_PATH + "mix.wav"

# Stem paths (if RUN_STEM_MASTERING = True)
STEM_PATHS = {
    "drums": BASE_INPUT_PATH + "drums.wav",
    "bass": BASE_INPUT_PATH + "bass.wav", 
    "vocals": BASE_INPUT_PATH + "vocals.wav",
    "music": BASE_INPUT_PATH + "music.wav"
}

# Stem Balance Control (applies to BIG variants)
STEM_GAINS = {
    "drums": 3.0,   # Drum level multiplier
    "bass": 2.8,    # Bass level multiplier
    "vocals": 4.0,  # Vocal level multiplier
    "music": 2.0    # Music level multiplier
}


# Output Control
PROJECT_NAME = "postmix_clean"
CREATE_INDIVIDUAL_STEM_FILES = False  # Set True to save processed stems separately

# Processing Options
MAX_RECOMMENDATIONS = 3  # Number of recommendation-based variants
ENABLE_STREAMING_SIMULATION = True  # Simulate streaming platform processing

print("✅ Configuration loaded!")
print(f"📁 Input: {BASE_INPUT_PATH}")
print(f"🎯 Mode: {'Stem' if RUN_STEM_MASTERING else 'Single-file'} processing")
if RUN_STEM_MASTERING:
    print(f"🎚️ Stem gains: Drums={STEM_GAINS['drums']}, Bass={STEM_GAINS['bass']}, Vocals={STEM_GAINS['vocals']}, Music={STEM_GAINS['music']}")

✅ Configuration loaded!
📁 Input: /Users/itay/Documents/post_mix_data/professional_mixes/10_configurations_20250901_120434/01_Standard_Mix/
🎯 Mode: Stem processing
🎚️ Stem gains: Drums=3.0, Bass=2.8, Vocals=4.0, Music=2.0


## 📦 Load Dependencies

In [2]:
# Core imports
import os
import json
import numpy as np
import soundfile as sf
from datetime import datetime

# Project modules - only what we actually use
from config import CONFIG
from data_handler import (
    make_workspace, Manifest, write_manifest,
    import_mix, register_input, load_wav, print_audio_summary
)
from analysis import analyze_wav
from render_engine import RenderEngine, StemRenderEngine, RenderOptions, PreprocessConfig
from mastering_orchestrator import MasteringOrchestrator, LocalMasterProvider
from stem_mastering import load_stem_set, validate_stem_set
from logging_versioning import (
    RunLogger, capture_environment, CODE_VERSIONS,
    register_and_log_artifact, make_repro_zip, json_sha256
)
from presets_recommendations import (
    recommend_from_analysis, build_all_variants_plan,
    recommendation_summary
)
from utils import ensure_audio_valid

print("✅ All dependencies loaded")
print(f"📊 Using CONFIG version: {CONFIG.to_dict()['audio']['default_bit_depth']}")

Post-Mix I/O layer loaded: AudioBuffer, load_wav, save_wav, resample_poly, slice_preview, with_suffix, auto_out_path, sha256_file, print_audio_summary.
Convenience helpers loaded: make_workspace, batch_load_wavs, env_fingerprint, Manifest, write/read_manifest, register_input, register_artifact, import_mix.
Analysis layer loaded: analyze_wav/analyze_audio_array, analysis_table, plot_spectrum, plot_short_term_loudness, plot_waveform_excerpt, LUFS approx, true-peak approx, stereo & health metrics.
DSP Primitives Layer loaded:
- Gain/level: apply_gain_db, normalize_peak, normalize_lufs, measure_peak, measure_rms
- Filters: highpass_filter, lowpass_filter, bandpass_filter, shelf_filter, peaking_eq, notch_filter, tilt_eq
- Stereo: mid_side_encode, mid_side_decode, stereo_widener
- Dynamics: compressor (soft‑knee), transient_shaper
- Fades: fade_in, fade_out
- K‑weighting/LUFS approx: k_weight, lufs_integrated_approx
Patched: render_from_cache now unpacks widen_stereo tuple correctly.
Process

## 🔧 Apply Configuration

In [3]:
# Apply stem gains to CONFIG if using stem mastering
if RUN_STEM_MASTERING:
    print("🎚️ Applying stem balance configuration...")
    
    # Update CONFIG with user's stem gains
    CONFIG.pipeline.stem_gains.update(STEM_GAINS)
    CONFIG.pipeline.create_individual_stem_files = CREATE_INDIVIDUAL_STEM_FILES
    
    print("✅ Stem gains applied to CONFIG:")
    for stem, gain in STEM_GAINS.items():
        print(f"   {stem}: {gain}x")
else:
    print("📄 Single-file mode - stem gains not applicable")

🎚️ Applying stem balance configuration...
✅ Stem gains applied to CONFIG:
   drums: 3.0x
   bass: 2.8x
   vocals: 4.0x
   music: 2.0x


## 📁 Setup Workspace & Import Audio

In [4]:
# Setup workspace
print("📁 Setting up workspace...")
workspace_paths = make_workspace(project=PROJECT_NAME)
manifest = Manifest(project=PROJECT_NAME, workspace=workspace_paths)
logger = RunLogger.start(workspace_paths.root, tag=PROJECT_NAME)
environment_info = capture_environment()
logger.log_event("env", {"environment": environment_info})

print(f"✅ Workspace: {workspace_paths.root}")

# Store data for each mode
single_file_data = None
stem_set_data = None

📁 Setting up workspace...
Workspace created at: /Users/itay/Documents/post_mix_data/PostMixRuns/postmix_clean_20250901-200541
✅ Workspace: /Users/itay/Documents/post_mix_data/PostMixRuns/postmix_clean_20250901-200541


In [5]:
# Import audio based on selected mode
if RUN_SINGLE_FILE:
    print("📄 Loading single file...")
    
    if not os.path.exists(MIX_SOURCE_PATH):
        raise FileNotFoundError(f"File not found: {MIX_SOURCE_PATH}")
    
    mix_path = import_mix(workspace_paths, MIX_SOURCE_PATH, alias="mix.wav")
    register_input(manifest, mix_path, alias="mix")
    
    audio_data, sample_rate = sf.read(mix_path)
    ensure_audio_valid(audio_data, "input mix")
    
    audio_buffer = load_wav(mix_path)
    print_audio_summary(audio_buffer, "Input Mix")
    
    single_file_data = {
        "audio_data": audio_data,
        "sample_rate": sample_rate,
        "mix_path": mix_path,
        "audio_buffer": audio_buffer
    }
    print("✅ Single file loaded")

if RUN_STEM_MASTERING:
    print("🎛️ Loading stems...")
    
    # Validate stem paths
    valid_stem_paths = {k: v for k, v in STEM_PATHS.items() if v and os.path.exists(v)}
    
    if not valid_stem_paths:
        raise ValueError("No valid stem files found")
    
    print(f"Found {len(valid_stem_paths)} stems: {', '.join(valid_stem_paths.keys())}")
    
    # Load stems
    stem_set = load_stem_set(valid_stem_paths)
    if not validate_stem_set(stem_set):
        raise ValueError("Stem validation failed")
    
    # Register stems
    for stem_type, stem_path in valid_stem_paths.items():
        imported = import_mix(workspace_paths, stem_path, alias=f"{stem_type}.wav")
        register_input(manifest, imported, alias=stem_type)
        register_and_log_artifact(manifest, logger, imported, kind="stem_input",
                                 params={"stem_type": stem_type}, stage="import")
    
    stem_set_data = {
        "stem_set": stem_set,
        "valid_stem_paths": valid_stem_paths,
        "sample_rate": 44100  # Standard rate
    }
    print(f"✅ {len(valid_stem_paths)} stems loaded")

📄 Loading single file...
Imported mix → /Users/itay/Documents/post_mix_data/PostMixRuns/postmix_clean_20250901-200541/inputs/mix.wav


/Users/itay/Documents/GitHub/post_mix_analysis/data_handler.py:160: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sr, data = wavfile.read(path)


Input Mix: sr=44100 | ch=2 | dur=358.0s | peak=0.966051 | rms=0.10874
  path: /Users/itay/Documents/post_mix_data/PostMixRuns/postmix_clean_20250901-200541/inputs/mix.wav
  sha256: f6e7843d5419f05a...
  src dtype: float32 | src ch: 2
✅ Single file loaded
🎛️ Loading stems...
Found 4 stems: drums, bass, vocals, music
✅ Loaded drums stem: 358.0s
✅ Loaded bass stem: 358.0s
✅ Loaded vocals stem: 358.0s
✅ Loaded music stem: 358.0s
✅ Found 4 active stems: drums, bass, vocals, music
✅ All stems at 44100 Hz
Imported mix → /Users/itay/Documents/post_mix_data/PostMixRuns/postmix_clean_20250901-200541/inputs/drums.wav
Imported mix → /Users/itay/Documents/post_mix_data/PostMixRuns/postmix_clean_20250901-200541/inputs/bass.wav
Imported mix → /Users/itay/Documents/post_mix_data/PostMixRuns/postmix_clean_20250901-200541/inputs/vocals.wav
Imported mix → /Users/itay/Documents/post_mix_data/PostMixRuns/postmix_clean_20250901-200541/inputs/music.wav
✅ 4 stems loaded


## 📊 Analysis & Planning

In [6]:
# Analyze audio and create processing plan
single_file_plan = None
stem_plan = None

if RUN_SINGLE_FILE and single_file_data:
    print("📊 Analyzing single file...")
    
    analysis = analyze_wav(single_file_data["mix_path"])
    
    # Generate recommendations
    recommendations = recommend_from_analysis(analysis)
    print(recommendation_summary(recommendations))
    
    # Build processing plan
    single_file_plan = build_all_variants_plan(
        recommendations, prefix="SF", top_recommendations=MAX_RECOMMENDATIONS
    )
    print(f"✅ Created {len(single_file_plan)} variants")

if RUN_STEM_MASTERING and stem_set_data:
    print("📊 Analyzing stems...")
    
    # Use configured stem combinations (BIG variants)
    stem_plan = CONFIG.pipeline.stem_combinations
    print(f"✅ Using {len(stem_plan)} BIG variant combinations")
    print(f"   Variants: {', '.join([combo[0] for combo in stem_plan[:5]])}...")

📊 Analyzing single file...


/Users/itay/Documents/GitHub/post_mix_analysis/analysis.py:285: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sr, data = wavfile.read(path)


- De-mud & Open Top (priority 1): B28 P30 C18 A25 W8
    • bass_energy %89.7 is high → reduce mud via clarity + keep lows controlled
    • air_energy %0.003 is low → add air shelf for brightness
    • crest 17.8 dB → moderate punch to emphasize transients
    ↪ Start here if the mix feels boomy/dull.
- Punch Up the Transients (priority 2): B16 P38 C10 A14 W8
    • crest 17.8 dB (dynamic) and LUFS -20.3 (quiet) → add transient definition
    • small air lift to help intelligibility
- Widen Image (Safe) (priority 4): B14 P14 C10 A12 W19
    • stereo_width 0.34 is narrow → add width
    • tone moves kept gentle to avoid destabilizing center
    ↪ If mono compatibility is critical, keep width ≤ 15.
✅ Created 39 variants
📊 Analyzing stems...
✅ Using 17 BIG variant combinations
   Variants: Reference_Mix, Standard_Mix, Drum_Heavy, Bass_Heavy, Vocal_Forward...


## 🎛️ Pre-Mastering Processing

In [7]:
# Process variants
all_variant_metadata = []
render_options = RenderOptions(
    target_peak_dbfs=CONFIG.audio.render_peak_target_dbfs,
    bit_depth=CONFIG.audio.default_bit_depth,
    hpf_hz=None,
    save_headroom_first=False
)

# Single-file processing
if RUN_SINGLE_FILE and single_file_plan:
    print("🔧 Processing single-file variants...")
    
    engine = RenderEngine(
        single_file_data["audio_data"],
        single_file_data["sample_rate"],
        preprocess=PreprocessConfig(
            low_cutoff=CONFIG.audio.prep_hpf_hz,
            kick_lo=CONFIG.audio.kick_freq_low,
            kick_hi=CONFIG.audio.kick_freq_high
        )
    )
    engine.preprocess()
    
    output_dir = os.path.join(workspace_paths.outputs, "single_file", "premasters")
    variant_metadata = engine.commit_variants(output_dir, single_file_plan, opts=render_options)
    
    for meta in variant_metadata:
        meta["processing_mode"] = "single_file"
    
    all_variant_metadata.extend(variant_metadata)
    print(f"✅ Created {len(variant_metadata)} single-file variants")

# Stem processing
if RUN_STEM_MASTERING and stem_plan:
    print("🎛️ Processing stem variants...")
    
    engine = StemRenderEngine(
        stem_set_data["stem_set"],
        preprocess=PreprocessConfig(
            low_cutoff=CONFIG.audio.prep_hpf_hz,
            kick_lo=CONFIG.audio.kick_freq_low,
            kick_hi=CONFIG.audio.kick_freq_high
        )
    )
    engine.preprocess_all_stems()
    
    output_dir = os.path.join(workspace_paths.outputs, "stem_mastering", "premasters")
    stem_results = engine.commit_stem_variants(output_dir, stem_plan, opts=render_options)
    
    # Convert to metadata format
    for variant_name, variant_data in stem_results.items():
        meta = {
            "out_path": variant_data["final_mix_path"],
            "variant_name": variant_name,
            "processing_mode": "stem_mastering"
        }
        all_variant_metadata.append(meta)
    
    print(f"✅ Created {len(stem_results)} stem variants")

print(f"\n📊 Total variants ready for mastering: {len(all_variant_metadata)}")
variant_metadata = all_variant_metadata

🔧 Processing single-file variants...
✅ Created 39 single-file variants
🎛️ Processing stem variants...
✅ Found 4 active stems: drums, bass, vocals, music
✅ All stems at 44100 Hz


KeyboardInterrupt: 

## 🎭 Mastering

In [8]:
# Apply mastering to all variants
print("🎭 Applying mastering...")

orchestrator = MasteringOrchestrator(workspace_paths, manifest)
provider = LocalMasterProvider(bit_depth="PCM_24")

master_results = []
master_paths = []

for i, variant_meta in enumerate(variant_metadata, 1):
    premaster_path = variant_meta["out_path"]
    
    if not os.path.exists(premaster_path):
        continue
    
    variant_name = variant_meta.get("variant_name", os.path.basename(premaster_path).replace(".wav", ""))
    print(f"[{i}/{len(variant_metadata)}] Mastering: {variant_name}")
    
    # Master with 4 styles
    results = orchestrator.run(
        premaster_path=premaster_path,
        providers=[provider],
        styles=None,  # Uses default 4 styles
        out_tag="masters",
        level_match_preview_lufs=-14.0
    )
    
    for result in results:
        master_results.append(result)
        master_paths.append(result.out_path)
        
        # Add -14 LUFS version
        lufs_file = os.path.join(
            os.path.dirname(result.out_path),
            f"{result.style}_-14LUFS.wav"
        )
        if os.path.exists(lufs_file):
            master_paths.append(lufs_file)

print(f"\n✅ Mastering complete!")
print(f"📁 Created {len(variant_metadata)} master folders")
print(f"🎵 Total files: {len(master_paths)}")

🎭 Applying mastering...
[1/56] Mastering: SF_Transparent
🎭 Creating mastered folder: SF_Transparent/ (8 files: 4 styles × 2 versions)
    Mastering input peak: -1.2 dBTP
    Normalized hot input down to -3.5 dBTP
    Skipped final normalization - peak at -3.3 dBTP is acceptable
      Applied limiting to prevent clipping (peak was 5.8 dBFS)
      loud -14 LUFS: -23.1 → -14.0 LUFS (gain: +9.1 dB)
    Mastering input peak: -1.2 dBTP
    Normalized hot input down to -3.5 dBTP
    Skipped final normalization - peak at -3.4 dBTP is acceptable
      Applied limiting to prevent clipping (peak was 5.9 dBFS)
      neutral -14 LUFS: -23.3 → -14.0 LUFS (gain: +9.3 dB)
    Mastering input peak: -1.2 dBTP
    Normalized hot input down to -3.5 dBTP
    Skipped final normalization - peak at -3.2 dBTP is acceptable
      Applied limiting to prevent clipping (peak was 5.8 dBFS)
      warm -14 LUFS: -23.1 → -14.0 LUFS (gain: +9.1 dB)
    Mastering input peak: -1.2 dBTP
    Normalized hot input down to -3

## 📦 Finalize & Create Bundle

In [9]:
# Create reproducibility bundle
print("📦 Creating reproducibility bundle...")

bundle_path = os.path.join(workspace_paths.reports, "bundles", f"{logger.run_id}.zip")

# Gather inputs
all_inputs = []
if single_file_data:
    all_inputs.append(single_file_data["mix_path"])
if stem_set_data:
    all_inputs.extend(list(stem_set_data["valid_stem_paths"].values()))

# Create bundle
reproducibility_zip = make_repro_zip(
    bundle_path,
    workspace_root=workspace_paths.root,
    run_logger=logger,
    env_info=environment_info,
    inputs=all_inputs,
    outputs=list(set(master_paths)),
    extra_jsons={
        "code_versions": CODE_VERSIONS,
        "configuration": CONFIG.to_dict(),
        "stem_gains": STEM_GAINS if RUN_STEM_MASTERING else None
    },
    readme_text="Post-mix processing with BIG variants and mastering"
)

# Write manifest
write_manifest(manifest)

# Log summary
logger.write_summary({
    "project": PROJECT_NAME,
    "processing_modes": {
        "single_file": RUN_SINGLE_FILE,
        "stem_mastering": RUN_STEM_MASTERING
    },
    "variants_created": len(variant_metadata),
    "masters_created": len(master_paths),
    "stem_gains": STEM_GAINS if RUN_STEM_MASTERING else None
})

print(f"✅ Bundle saved: {bundle_path}")

📦 Creating reproducibility bundle...


FileNotFoundError: [Errno 2] No such file or directory: '/Users/itay/Documents/post_mix_data/professional_mixes/mix_20250901_092454/drums.wav'

## 🎉 Summary

In [ ]:
# Final summary
print("="*60)
print("🎉 PROCESSING COMPLETE!")
print("="*60)
print(f"📁 Workspace: {workspace_paths.root}")
print()

if RUN_SINGLE_FILE:
    single_count = len([v for v in variant_metadata if v.get("processing_mode") == "single_file"])
    print(f"📄 Single-file: {single_count} variants")
    print(f"   Location: {workspace_paths.root}/outputs/single_file/")

if RUN_STEM_MASTERING:
    stem_count = len([v for v in variant_metadata if v.get("processing_mode") == "stem_mastering"])
    print(f"🎛️ Stem mastering: {stem_count} variants")
    print(f"   Location: {workspace_paths.root}/outputs/stem_mastering/")
    print(f"   Stem gains applied: {STEM_GAINS}")

print()
print(f"📊 Total variants: {len(variant_metadata)}")
print(f"🎵 Total master files: {len(master_paths)}")
print(f"📦 Bundle: {bundle_path}")
print()
print("✨ Ready for listening!")